# LangChain Hub

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Get a promt from the Hub...

In [3]:
from langchain import hub

# pull a chat prompt
prompt = hub.pull("efriis/my-first-prompt")
print(prompt)

input_variables=['profession', 'question'] metadata={'lc_hub_owner': 'efriis', 'lc_hub_repo': 'my-first-prompt', 'lc_hub_commit_hash': '56489e79537fc477d8368e6c9902df15b5e9fe8bc0e4f38dc4b15b65e550077c'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['profession'], template='You are an expert {profession} who loves answering questions cheerfully.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question} '))]


...and try it out:

In [4]:
# create a model to use it with
from langchain_openai import ChatOpenAI
model = ChatOpenAI()

# use it in a runnable
runnable = prompt | model
response = runnable.invoke({
	"profession": "biologist",
	"question": "What is special about parrots?",
})

print(response)

content='Oh, parrots are amazing creatures with so many unique qualities! One of the most special things about them is their ability to mimic human speech and sounds. They have a complex vocal system that allows them to imitate a wide range of sounds with incredible accuracy. Additionally, parrots are highly intelligent birds known for their problem-solving skills and social nature. They form strong bonds with their owners and have vibrant, colorful plumage that makes them truly stand out in the bird world. Overall, parrots are fascinating and charismatic animals that never fail to capture our attention!' response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 31, 'total_tokens': 143}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4eafe093-c72f-4087-8ebf-d6f48860bcac-0'


now get a more interesting one:

In [8]:
prompt = hub.pull("borislove/customer-sentiment-analysis")
print(prompt)

runnable = prompt | model
response = runnable.invoke({
	"client_letter": "I cannot cope with your fucking product. I will never buy anything again from you!",
    "format_instructions": ""
})

print(response)
print(response.content)

input_variables=['client_letter', 'format_instructions'] metadata={'lc_hub_owner': 'borislove', 'lc_hub_repo': 'customer-sentiment-analysis', 'lc_hub_commit_hash': '45eafd27466db2b27a448e29132a77a88a53092b7d4ad09838d13ef23908c36f'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="As a customer service representative, you receive the following email from a customer.\nYour task is to identify the customer's sentiment and categorize it based on the scale below:\n    0 - Calm: Customer asks questions but does not seem upset; is just seeking information.\n    1 - Slightly Frustrated: Customer shows subtle signs of irritation but is still open to solutions.\n    2 - Frustrated: Customer explicitly states being unhappy or irritated but is willing to discuss a solution.\n    3 - Very Frustrated: Customer is clearly agitated, uses strong language, or mentions the problem repeatedly.\n    4 - Extremely Frustrated: Customer is intensely unhappy, may raise 

## Output Parser

In [16]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

#model = ChatOpenAI() # GTP-3 does not return the exact format -> ValueError on parse
model = ChatOpenAI(model="gpt-4")

class EmotionalState(Enum):
    """Enum for the emotional state of the client"""
    CALM = "Calm"
    LIGHTLY_FRUSTRATED = "Lightly Frustrated"
    FRUSTRATED = "Frustrated"
    VERY_FRUSTRATED = "Very Frustrated"
    EXTREMELY_FRUSTRATED = "4 - Extremely Frustrated" # otherwise throws an ValueError
    OVERWHELMED = "Overwhelmed"
    UNKNOWN = "Unknown"

prompt = hub.pull("borislove/customer-sentiment-analysis")
print(prompt)

runnable = prompt | model
response = runnable.invoke({
	"client_letter": "I cannot cope with your fucking product. I will never buy anything again from you!",
    "format_instructions": ""
})

parser = EnumOutputParser(enum=EmotionalState)

# Parse the response to get the Enum result
print(response)
result = parser.parse(response.content)
print(result)

input_variables=['client_letter', 'format_instructions'] metadata={'lc_hub_owner': 'borislove', 'lc_hub_repo': 'customer-sentiment-analysis', 'lc_hub_commit_hash': '45eafd27466db2b27a448e29132a77a88a53092b7d4ad09838d13ef23908c36f'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="As a customer service representative, you receive the following email from a customer.\nYour task is to identify the customer's sentiment and categorize it based on the scale below:\n    0 - Calm: Customer asks questions but does not seem upset; is just seeking information.\n    1 - Slightly Frustrated: Customer shows subtle signs of irritation but is still open to solutions.\n    2 - Frustrated: Customer explicitly states being unhappy or irritated but is willing to discuss a solution.\n    3 - Very Frustrated: Customer is clearly agitated, uses strong language, or mentions the problem repeatedly.\n    4 - Extremely Frustrated: Customer is intensely unhappy, may raise 

versteht er auch deutsch...?

In [18]:
response = runnable.invoke({
	"client_letter": "Ich komme mit ihrem Produkt überhaupt nicht zurecht! Ich bin verzweifelt, dass es so ein schlechtes Produkt ist. Ich würde es nicht mehr kaufen!",
    "format_instructions": ""
})

parser = EnumOutputParser(enum=EmotionalState)

# Parse the response to get the Enum result
print(response)
#result = parser.parse(response.content)
#print(result)

content='4 - Extremely Frustrated: Customer is intensely unhappy, may raise their voice or use aggressive language.' response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 256, 'total_tokens': 278}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f148c6b9-7dd9-46fc-adcd-6168ecba579d-0'
